## NLP Final Project
Text to Text Generation Models: Seq2Seq and T5

Bias Classification Models: BiLSTM CNN, BERT CNN, and DistilRoberta

# Text to Text Generation Model Analysis

Sequence to Sequence Transformer Model

In [ ]:
!pip install keras-nlp

In [ ]:
#set up
import pandas as pd
import keras_nlp
import keras
from keras.models import load_model
import tensorflow as tf
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [11]:
BATCH_SIZE = 64
EPOCHS = 15  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 256

#The size of our source and target language vocabularies
ORG_VOCAB_SIZE = 15000
MOD_VOCAB_SIZE = 15000

#define some hyperparameter values for our transformers
EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

# Encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ORG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    #mask_zero=True,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=MOD_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    #mask_zero=True,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(MOD_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

#connect the encoder and decoder together in sequence
seq2seq = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="s2sTransformer",
)

seq2seq.summary()

Model: "s2sTransformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, None, 256)      │      3,905,536 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, None, 15000)    │      9,339,288 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,559,896 (55.54 MB)

 Trainable params: 14,559,896 (55.54 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
seq2seq.load_weights("drive/MyDrive/seq2seq.weights.h5")

In [4]:
#test set
from google.colab import drive
drive.mount('/content/drive')
test_pairs = pd.read_csv('drive/MyDrive/test_pairsS2S.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
test_pairs.head()

,Unnamed: 0,text_pairs_dict
0,0,('he devoted his enormous energies to the dest...
1,1,"(', ehud goldwasser and eldad regev, in exchan..."
2,2,"('after being exposed all day on august 10, 19..."
3,3,('george c. scott later won (within the fictio...
4,4,('kim pyong-il (born 1954) is the half-brother...


In [5]:
# Example test data
test_source_sequences = test_pairs.text_pairs_dict.apply(lambda x: eval(x)).apply(lambda x: x[0])  # List of source sequences
test_target_sequences = test_pairs.text_pairs_dict.apply(lambda x: eval(x)).apply(lambda x: x[1])   # List of target sequences

In [60]:
test_source_sequences[0]

'he devoted his enormous energies to the destruction of what he considered the slave power, that is the conspiracy he saw of slave owners to seize control of the federal government and block the progress of liberty .'

In [8]:
!pip install rouge

In [6]:
test_s2s_sequences1 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences.csv')
test_s2s_sequences2 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences2.csv')
test_s2s_sequences3 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences3.csv')

In [7]:
test_s2s_sequences = test_s2s_sequences1.iloc[:, 1].to_list() + test_s2s_sequences2.iloc[:, 1].to_list() + test_s2s_sequences3.iloc[:, 1].to_list()

In [11]:
test_target_sequences_s2s = test_target_sequences

In [18]:
#evaluate metrics

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from sklearn.metrics import accuracy_score
from difflib import SequenceMatcher

# Compute BLEU score
bleu_scores = [sentence_bleu([target_sequence], predicted_sequence) for target_sequence, predicted_sequence in zip(test_target_sequences_s2s, test_s2s_sequences)]
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
overall_bleu_score = corpus_bleu([[target_sequence] for target_sequence in test_target_sequences_s2s], test_s2s_sequences)

# Compute ROUGE score
rouge = Rouge()
rouge_scores = rouge.get_scores([predicted_sequence for predicted_sequence in test_s2s_sequences], [target_sequence for target_sequence in test_target_sequences_s2s], avg=True)

# Compute accuracy
accuracy = accuracy_score(test_target_sequences_s2s, test_s2s_sequences)

# Compute Similar Sequence Matcher score
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similarity_score = similar(test_target_sequences_s2s, test_s2s_sequences)

# Print or store the evaluation metrics
print("BLEU Score (Average):", average_bleu_score)
print("BLEU Score (Overall):", overall_bleu_score)
print("ROUGE Score (Avg):", rouge_scores)
print("Accuracy:", accuracy)
print("Similarity:", similarity_score)


BLEU Score (Average): 0.6694707256875928
BLEU Score (Overall): 0.7107922905503329
ROUGE Score (Avg): {'rouge-1': {'r': 0.6464857870450086, 'p': 0.693310098649049, 'f': 0.6620201535814505}, 'rouge-2': {'r': 0.44568326070159725, 'p': 0.44206707645953525, 'f': 0.4378814663619914}, 'rouge-l': {'r': 0.6274521961258638, 'p': 0.6718451612054863, 'f': 0.6420638286630264}}
Accuracy: 0.0005714285714285715
Similarity: 0.0005714285714285715


T5 Fine Tuned Model

In [1]:
!pip install tensorflow==2.9.0

In [2]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [3]:
# Load the pretrained tensorflow model

model_name = 't5-base'
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = TFT5ForConditionalGeneration.from_pretrained(model_name)
max_length = 128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [6]:
def build_t5_training_wrapper_model(t5_model, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = t5_model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [7]:
# Trained in separate notebook
# Load model
checkpoint_dir = 'drive/MyDrive/'
checkpoint_filepath = checkpoint_dir + 't5_bias_weights.01-0.98.hdf5'

model_wrapper = build_t5_training_wrapper_model(t5_model, max_length)
model_wrapper.load_weights(checkpoint_filepath)

In [6]:
# Test examples
prefix = 'translate biased to unbiased: '

for test_input_text in ['The player must not make any move that would place his king in check.',
                        "The lyrics are about mankind's perceived idea of hell.",
                        'Marriage is a holy union of individuals.']:
    test_inputs = t5_tokenizer([prefix + test_input_text], return_tensors='tf')
    test_output_ids = t5_model.generate(test_inputs['input_ids'], max_length=128)

    print([t5_tokenizer.decode(out_ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False) for out_ids in test_output_ids])


['the player must not make any move that would place their king in check.']
["the lyrics are about humankind's perceived idea of hell."]
['marriage is a union of individuals.']


In [8]:
test_t5_sequences3 = []

# Prepare inputs for batch processing
prefix = 'translate biased to unbiased: '
batch_size = 500
start_index = 8506
num_batches = (len(test_source_sequences) - start_index + batch_size - 1) // batch_size

# Process batches
for i in range(num_batches):
    start_idx = start_index + i * batch_size
    end_idx = min(start_index + (i + 1) * batch_size, len(test_source_sequences))
    batch_sequences = test_source_sequences[start_idx:end_idx]

    # Tokenize batch inputs
    batch_inputs = [prefix + source_sequence for source_sequence in batch_sequences]
    test_inputs = t5_tokenizer(batch_inputs, padding=True, return_tensors='tf')

    # Generate batch outputs
    test_output_ids = t5_model.generate(test_inputs.input_ids, max_length=128)
    predicted_sequences = t5_tokenizer.batch_decode(test_output_ids, skip_special_tokens=True)

    # Append batch outputs to results list
    test_t5_sequences3.extend(predicted_sequences)

# Ensure the number of generated sequences matches the number of input sequences
assert len(test_t5_sequences3) == len(test_source_sequences) - start_index


In [10]:
pd.DataFrame(test_t5_sequences3).to_csv("drive/MyDrive/test_t5_sequences3.csv")

In [5]:
test_t5_sequences1 = pd.read_csv('drive/MyDrive/test_t5_sequences.csv')
test_t5_sequences2 = pd.read_csv('drive/MyDrive/test_t5_sequences2.csv')
test_t5_sequences3 = pd.read_csv('drive/MyDrive/test_t5_sequences3.csv')

In [6]:
test_t5_sequences = test_t5_sequences1.iloc[:, 1].to_list() + test_t5_sequences2.iloc[:, 1].to_list() + test_t5_sequences3.iloc[:, 1].to_list()

In [28]:
!pip install rouge

In [36]:
#evaluate metrics

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from sklearn.metrics import accuracy_score
from difflib import SequenceMatcher

# Compute BLEU score
bleu_scores = [sentence_bleu([target_sequence], predicted_sequence) for target_sequence, predicted_sequence in zip(test_target_sequences, test_t5_sequences)]
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
overall_bleu_score = corpus_bleu([[target_sequence] for target_sequence in test_target_sequences], test_t5_sequences)

# Compute ROUGE score
rouge = Rouge()
rouge_scores = rouge.get_scores([predicted_sequence for predicted_sequence in test_t5_sequences], [target_sequence for target_sequence in test_target_sequences], avg=True)

# Compute accuracy
accuracy = accuracy_score(test_target_sequences, test_t5_sequences)

# Compute Similar Sequence Matcher score
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similarity_score = similar(test_target_sequences, test_t5_sequences)

# Print or store the evaluation metrics
print("BLEU Score (Average):", average_bleu_score)
print("BLEU Score (Overall):", overall_bleu_score)
print("ROUGE Score (Avg):", rouge_scores)
print("Accuracy:", accuracy)
print("Similarity:", similarity_score)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU Score (Average): 0.878263707859452
BLEU Score (Overall): 0.9192834457440578
ROUGE Score (Avg): {'rouge-1': {'r': 0.9092093582715605, 'p': 0.9126912072614415, 'f': 0.9072120316154018}, 'rouge-2': {'r': 0.8540940757006562, 'p': 0.8560329211879716, 'f': 0.850897311165589}, 'rouge-l': {'r': 0.9060885884780415, 'p': 0.9094271244054254, 'f': 0.904057874984002}}
Accuracy: 0.25867009551800146
Similarity: 0.25867009551800146


# Bias Classification of Text Generation Models


Bias Classification
-for each bias classification model, measure bias of each model's newly generated sentences

DistilRoberta Bias Classification

In [1]:
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("valurank/distilroberta-bias")
model = AutoModelForSequenceClassification.from_pretrained("valurank/distilroberta-bias")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [2]:
#example sentences
print(classifier("A lead programmer usually spends his career mired in obscurity."))
print(classifier("A lead programmer usually spends their career mired in obscurity."))
print(classifier("The lyrics are about mankind's perceived idea of hell."))
print(classifier("The lyrics are about humanity's perceived idea of hell."))
print(classifier("Marriage is a holy union of individuals."))
print(classifier("Marriage is a personal union of individuals."))


[{'label': 'NEUTRAL', 'score': 0.5882661938667297}]
[{'label': 'NEUTRAL', 'score': 0.6250182390213013}]
[{'label': 'NEUTRAL', 'score': 0.6530860662460327}]
[{'label': 'NEUTRAL', 'score': 0.7113867402076721}]
[{'label': 'BIASED', 'score': 0.9932663440704346}]
[{'label': 'BIASED', 'score': 0.6571064591407776}]


In [3]:
from google.colab import drive
drive.mount('/content/drive')
#test set
import pandas as pd
test_pairs = pd.read_csv('drive/MyDrive/test_pairsS2S.csv')

Mounted at /content/drive


In [4]:
# Example test data
test_source_sequences = test_pairs.text_pairs_dict.apply(lambda x: eval(x)).apply(lambda x: x[0])  # List of source sequences
test_target_sequences = test_pairs.text_pairs_dict.apply(lambda x: eval(x)).apply(lambda x: x[1])   # List of target sequences

In [15]:
#original test neutral bias scores
cls_test_source = classifier([sentence for sentence in test_source_sequences])

In [16]:
source_sums = {label: sum(item['score'] for item in cls_test_source if item['label'] == label) for label in set(item['label'] for item in cls_test_source)}
# Print the sums for each label group
print("Sum of scores for each label group for Source Sentences:")
for label, score_sum in source_sums.items():
    print(f"{label}: {score_sum}")

Sum of scores for each label group for Source Sentences:
NEUTRAL: 8506.840076565742
BIASED: 12965.265796661377


In [19]:
neutral_count = sum(1 for item in cls_test_source if item['label'] == 'NEUTRAL')

# Print the count of labeled neutral instances
print("Number of labeled neutral instances:", neutral_count)

Number of labeled neutral instances: 11735


In [23]:
Biased_source = (11735 - 8506.840076565742 + 12965.265796661377) / 27220
Biased_source

0.5949091006647919

In [17]:
#target test neutral bias scores
cls_test_target = classifier([sentence for sentence in test_target_sequences])

In [18]:
target_sums = {label: sum(item['score'] for item in cls_test_target if item['label'] == label) for label in set(item['label'] for item in cls_test_target)}
# Print the sums for each label group
print("Sum of scores for each label group for Target Sentences:")
for label, score_sum in target_sums.items():
    print(f"{label}: {score_sum}")

Sum of scores for each label group for Target Sentences:
NEUTRAL: 17220.591447412968
BIASED: 4128.140566170216


In [20]:
neutral_count = sum(1 for item in cls_test_target if item['label'] == 'NEUTRAL')

# Print the count of labeled neutral instances
print("Number of labeled neutral instances:", neutral_count)

Number of labeled neutral instances: 21442


In [22]:
Biased_target = (21442 - 17220.591447412968 + 4128.140566170216) / 27220
Biased_target

0.30674317115199295

In [7]:
#t5 generated test neutral bias scores
cls_test_t5 = classifier([sentence for sentence in test_t5_sequences])

In [8]:
t5_sums = {label: sum(item['score'] for item in cls_test_t5 if item['label'] == label) for label in set(item['label'] for item in cls_test_t5)}
# Print the sums for each label group
print("Sum of scores for each label group for T5 Generated Sentences:")
for label, score_sum in t5_sums.items():
    print(f"{label}: {score_sum}")

Sum of scores for each label group for T5 Generated Sentences:
BIASED: 4972.125776350498
NEUTRAL: 16335.142832159996


In [9]:
neutral_count = sum(1 for item in cls_test_t5 if item['label'] == 'NEUTRAL')

# Print the count of labeled neutral instances
print("Number of labeled neutral instances:", neutral_count)

Number of labeled neutral instances: 20364


In [10]:
Biased_t5 = (20364 - 16335.142832159996 + 4972.125776350498) / 27220
Biased_t5

0.3306753469577701

In [ ]:
#seq2seq generated test neutral bias scores
cls_test_seq2seq = classifier([sentence for sentence in test_seq2seq_sequences])

In [ ]:
seq2seq_sums = {label: sum(item['score'] for item in cls_test_seq2seq if item['label'] == label) for label in set(item['label'] for item in cls_test_seq2seq)}
# Print the sums for each label group
print("Sum of scores for each label group for Seq2Seq Generated Sentences:")
for label, score_sum in seq2seq_sums.items():
    print(f"{label}: {score_sum}")

In [ ]:
neutral_count = sum(1 for item in cls_test_seq2seq if item['label'] == 'NEUTRAL')

# Print the count of labeled neutral instances
print("Number of labeled neutral instances:", neutral_count)

# Generate on New Dataset

In [ ]:
df = pd.read_csv('drive/MyDrive/news-article-categories.csv')
df.head()

In [ ]:
df.body = df.body.str.replace('\xa0', ' ')

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


output_articles = []

for input_article in df.body:
    input_sentences = sent_tokenize(input_article)
    output_article = ""
    for input_sentence in input_sentences:
      inputs = t5_tokenizer([prefix + input_sentence], return_tensors='tf')
      output_ids = t5_model.generate(inputs['input_ids'], max_length=max_length)
      output_sentence = [t5_tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out_ids in output_ids]
      output_article += output_sentence + " "
    output_articles.append(output_article)

df["neutral_body"] = output_articles

In [ ]:
c = "The New Yorker is taking on President Donald Trump after he asked why the United States would welcome immigrants from 'shithole' places like Haiti and African countries during a bipartisan Oval Office meeting on Thursday. 'In the Hole,' by artist Anthony Russo, responds to the president's comment, which has been decried as racist by the United Nations, with a stark illustration for the magazine's cover in its Jan. 22 issue. On a field of white, the president's yellowish sweep of hair is just visible from the depths of a black hole: On Thursday, Trump sparked widespread criticism after he reportedly questioned why the U.S. should restore."

# Split the text into sentences using NLTK's sent_tokenize
sentences = sent_tokenize(c)

# Initialize an empty string to store the generated paragraph
generated_paragraph = ""

# Generate text for each input sentence
for test_input_text in sentences:
    test_inputs = t5_tokenizer([prefix + test_input_text], return_tensors='tf', padding=True, truncation=True, max_length=512)
    test_output_ids = t5_model.generate(test_inputs.input_ids, max_length=max_length)

    # Decode and concatenate the generated text to the paragraph
    generated_sentence = t5_tokenizer.decode(test_output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    generated_paragraph += generated_sentence + " "

# Print or save the generated paragraph
print(generated_paragraph)


In [ ]:
sent_tokenize(c)